In [1]:

from Core.Tensor import Tensor
from Core.Operation import *
from Core.Model import *
from Core.Loss import *
from Core.Optimizer import *
import keras
class TwoLayerModel(Model):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.linear1 = Linear(784, 128)
        self.linear2 = Linear(128, 10)
        self.relu = Relu()
        self.softmax = Softmax(dim=1)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


# x_train = x_train.reshape(-1, 784)
# x_test = x_test.reshape(-1, 784)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [2]:


model = TwoLayerModel()
loss_fn = MSE()
optimizer = SGD(model.get_parameters(), lr=0.001)

batch_size = 128
epochs = 20

for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        batch_x = x_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        batch_x = Tensor(batch_x)
        batch_y = Tensor(batch_y)
        
        output = model(batch_x)
        
        loss = loss_fn(output, batch_y)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    output = model(x_test)
    acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
    print("Epoch: %d, Loss: %.5f, Accuracy: %.5f" % (epoch+1, loss.data, acc))

Epoch: 1, Loss: 0.01510, Accuracy: 0.89020
Epoch: 2, Loss: 0.01161, Accuracy: 0.90600
Epoch: 3, Loss: 0.01010, Accuracy: 0.91600
Epoch: 4, Loss: 0.00911, Accuracy: 0.92110
Epoch: 5, Loss: 0.00834, Accuracy: 0.92800
Epoch: 6, Loss: 0.00772, Accuracy: 0.93340
Epoch: 7, Loss: 0.00726, Accuracy: 0.93720
Epoch: 8, Loss: 0.00689, Accuracy: 0.94000
Epoch: 9, Loss: 0.00658, Accuracy: 0.94350
Epoch: 10, Loss: 0.00629, Accuracy: 0.94560
Epoch: 11, Loss: 0.00604, Accuracy: 0.94650
Epoch: 12, Loss: 0.00582, Accuracy: 0.94920
Epoch: 13, Loss: 0.00561, Accuracy: 0.95120
Epoch: 14, Loss: 0.00545, Accuracy: 0.95390
Epoch: 15, Loss: 0.00531, Accuracy: 0.95470
Epoch: 16, Loss: 0.00517, Accuracy: 0.95630
Epoch: 17, Loss: 0.00506, Accuracy: 0.95780
Epoch: 18, Loss: 0.00496, Accuracy: 0.95910
Epoch: 19, Loss: 0.00487, Accuracy: 0.96020
Epoch: 20, Loss: 0.00477, Accuracy: 0.96070
